# Part 1
## The brute-force sign flip algorithm for encoding classical inputs in quantum state vectors

Inspired by work by Francesco Tacchino, Chiara Macchiavello, Dario Gerace & Daniele Bajoni https://www.nature.com/articles/s41534-019-0140-4?amp%3Bcode=4cf1b507-7e23-4df0-a2a1-e82a3fe2bc4b

In [1]:
import math
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import BasicAer

## Generate all possible binary-valued vectors in 4 dimensions (2 qubits)

A quantum state vector of N=2 qubits is described by a vector of dimension $m=2^N=4$. With binary amplitudes, this implies $2^{2^N}=16$ distinct states. We will use {-1,1} as components of our binary vector.
<br><br>
A way to think about generating the binary vectors is to assign an interger $k$ to each vector. You can convert the integer $k$ to a four-digit binary string ({0,1}) $n_0 n_1 n_2 n_3$ and gernate the vectors as $$\vec{v} = [(-1)^{n_0}, (-1)^{n_1}, (-1)^{n_2}, (-1)^{n_3}]$$

In [2]:
def k2vec(k, m):
    """
        Parameters: 
            - k (integer) the integer number for the corresponding binary vector
            - m (integer) the length of binary string (needed to distinguish all possibilities)
        Returns: 
            - v (list) a binary vector correspoding to integer k 
        
        Description: By taking a fixed total ordering of binary strings of fixed length we
        can associate each binary vector with an integer. The integer is converted to a 
        binary string which can be used to generate unique vectors.
    """
    
    v = -1*np.ones(m)
    binary_string = ("{:0%db}"%m).format(k) # convert k to an m-digit binary number
    #print(binary_string)
    v = list(map(lambda v, b : v**int(b), v, binary_string)) 
    
    return v

In [3]:
m = 4 # for two qubits
vectors = [k2vec(k,m) for k in range(16)]
for i, v in enumerate(vectors):
    print("k=",i," ", v)

k= 0   [1.0, 1.0, 1.0, 1.0]
k= 1   [1.0, 1.0, 1.0, -1.0]
k= 2   [1.0, 1.0, -1.0, 1.0]
k= 3   [1.0, 1.0, -1.0, -1.0]
k= 4   [1.0, -1.0, 1.0, 1.0]
k= 5   [1.0, -1.0, 1.0, -1.0]
k= 6   [1.0, -1.0, -1.0, 1.0]
k= 7   [1.0, -1.0, -1.0, -1.0]
k= 8   [-1.0, 1.0, 1.0, 1.0]
k= 9   [-1.0, 1.0, 1.0, -1.0]
k= 10   [-1.0, 1.0, -1.0, 1.0]
k= 11   [-1.0, 1.0, -1.0, -1.0]
k= 12   [-1.0, -1.0, 1.0, 1.0]
k= 13   [-1.0, -1.0, 1.0, -1.0]
k= 14   [-1.0, -1.0, -1.0, 1.0]
k= 15   [-1.0, -1.0, -1.0, -1.0]


We can encode this input vector as the quantum state vector $|\Psi_i>$ such that $$|\Psi_i> = \frac{1}{\sqrt(m)} \sum_{j=0}^{m-1}i_j|j>$$
Where $m$ is the dimensionality of the input, $i_j$ is an element of the classical input vector, and $|j>$ is a computational basis state.

In [4]:
state_vectors =  [v/np.sqrt(len(v)) for v in vectors]
for i, v in enumerate(state_vectors):
    print("k=",i," ", v)

k= 0   [0.5 0.5 0.5 0.5]
k= 1   [ 0.5  0.5  0.5 -0.5]
k= 2   [ 0.5  0.5 -0.5  0.5]
k= 3   [ 0.5  0.5 -0.5 -0.5]
k= 4   [ 0.5 -0.5  0.5  0.5]
k= 5   [ 0.5 -0.5  0.5 -0.5]
k= 6   [ 0.5 -0.5 -0.5  0.5]
k= 7   [ 0.5 -0.5 -0.5 -0.5]
k= 8   [-0.5  0.5  0.5  0.5]
k= 9   [-0.5  0.5  0.5 -0.5]
k= 10   [-0.5  0.5 -0.5  0.5]
k= 11   [-0.5  0.5 -0.5 -0.5]
k= 12   [-0.5 -0.5  0.5  0.5]
k= 13   [-0.5 -0.5  0.5 -0.5]
k= 14   [-0.5 -0.5 -0.5  0.5]
k= 15   [-0.5 -0.5 -0.5 -0.5]


## What quantum circuits will encode these vectors?
Think of a quantum circuit as the program. With a quantum computer we don't have easy access to read in/out so we have to change our quantum circuits (change our program) to run with different inputs. It's like assigning values to variables at the top of a program.
<br><br>
We'll use a "brute-force Sign-Flip" algorithm. The idea is that for entry of -1 in the input vector you apply a -1 factor to the corresponding entry in the quantum state vector.
<br><br>
The only gates the brute-force Sign-Flip algorithm uses only X and CZ gates. The X gate takes $|0> \rightarrow |1>$ and $|1> \rightarrow |0>$. The CZ has the effect of switching the sign of |11>.

In [5]:
def draw_state_vector(circ):
    """
        Input: 
            - circ (qiskit.circuit.quantumcircuit.QuantumCircuit) quantum circuit 
              to be executed
        Description:
            - Runs the quantum circuit and prints the resulting quantum state vector 
              and draws the circuit.
    """
    
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circ, backend)
    result = job.result()
    output_state = result.get_statevector(circ, decimals=3)
    print("state vector after circuit:", output_state)
    print(circ.draw())

In [6]:
# create quantum register and circuit
q = QuantumRegister(2, 'q')
circ = QuantumCircuit(q)

Parallel hadamard gates will produce an equal super position, $|00> \rightarrow |+>^{\otimes 2}$
<br><br>
We will need this at the start of every circuit. It also produces the corresponding state vector for k=0!

In [7]:
k=0
print("target state vector:", state_vectors[k])
circ.h(q[0])
circ.h(q[1])

draw_state_vector(circ)

target state vector: [0.5 0.5 0.5 0.5]
state vector after circuit: [0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]
             ┌───┐
q_0: |0>─────┤ H ├
        ┌───┐└───┘
q_1: |0>┤ H ├─────
        └───┘     


In [8]:
k=1
print("target state vector:", state_vectors[k])
circ1 = QuantumCircuit(q)
circ1.cz(q[0], q[1])

draw_state_vector(circ + circ1)

target state vector: [ 0.5  0.5  0.5 -0.5]
state vector after circuit: [ 0.5+0.j  0.5+0.j  0.5+0.j -0.5+0.j]
             ┌───┐   
q_0: |0>─────┤ H ├─■─
        ┌───┐└───┘ │ 
q_1: |0>┤ H ├──────■─
        └───┘        


In [9]:
k=2
print("target state vector:", state_vectors[k])

circ2 = QuantumCircuit(q)
circ2.x(q[0])
circ2.cz(q[0], q[1])
circ2.x(q[0])

draw_state_vector(circ + circ2)

target state vector: [ 0.5  0.5 -0.5  0.5]
state vector after circuit: [ 0.5+0.j  0.5+0.j -0.5+0.j  0.5+0.j]
             ┌───┐┌───┐   ┌───┐
q_0: |0>─────┤ H ├┤ X ├─■─┤ X ├
        ┌───┐└───┘└───┘ │ └───┘
q_1: |0>┤ H ├───────────■──────
        └───┘                  


In [10]:
k=3
print("target state vector:", state_vectors[k])
circ3 = QuantumCircuit(q)

circ3.x(q[0])
circ3.cz(q[0], q[1])
circ3.x(q[0])

circ3.cz(q[0], q[1])

draw_state_vector(circ + circ3)

target state vector: [ 0.5  0.5 -0.5 -0.5]
state vector after circuit: [ 0.5+0.j  0.5+0.j -0.5+0.j -0.5+0.j]
             ┌───┐┌───┐   ┌───┐   
q_0: |0>─────┤ H ├┤ X ├─■─┤ X ├─■─
        ┌───┐└───┘└───┘ │ └───┘ │ 
q_1: |0>┤ H ├───────────■───────■─
        └───┘                     


In [11]:
k=4
print("target state vector:", state_vectors[k])

circ4 = QuantumCircuit(q)
circ4.x(q[1])
circ4.cz(q[0], q[1])
circ4.x(q[1])

draw_state_vector(circ + circ4)

target state vector: [ 0.5 -0.5  0.5  0.5]
state vector after circuit: [ 0.5+0.j -0.5+0.j  0.5+0.j  0.5+0.j]
                  ┌───┐        
q_0: |0>──────────┤ H ├─■──────
        ┌───┐┌───┐└───┘ │ ┌───┐
q_1: |0>┤ H ├┤ X ├──────■─┤ X ├
        └───┘└───┘        └───┘


In [12]:
k=5
print("target state vector:", state_vectors[k])
circ5 = QuantumCircuit(q)

circ5.x(q[1])
circ5.cz(q[0], q[1])
circ5.x(q[1])

circ5.cz(q[0], q[1])

draw_state_vector(circ + circ5)

target state vector: [ 0.5 -0.5  0.5 -0.5]
state vector after circuit: [ 0.5+0.j -0.5+0.j  0.5+0.j -0.5+0.j]
                  ┌───┐           
q_0: |0>──────────┤ H ├─■───────■─
        ┌───┐┌───┐└───┘ │ ┌───┐ │ 
q_1: |0>┤ H ├┤ X ├──────■─┤ X ├─■─
        └───┘└───┘        └───┘   


In [13]:
k=6
print("target state vector:", state_vectors[k])

circ6 = QuantumCircuit(q)
circ6.x(q[0])
circ6.cz(q[0], q[1])
circ6.x(q[0])

circ6.x(q[1])
circ6.cz(q[0], q[1])
circ6.x(q[1])

draw_state_vector(circ + circ6)

target state vector: [ 0.5 -0.5 -0.5  0.5]
state vector after circuit: [ 0.5+0.j -0.5+0.j -0.5+0.j  0.5+0.j]
             ┌───┐┌───┐        ┌───┐        
q_0: |0>─────┤ H ├┤ X ├─■──────┤ X ├─■──────
        ┌───┐└───┘└───┘ │ ┌───┐└───┘ │ ┌───┐
q_1: |0>┤ H ├───────────■─┤ X ├──────■─┤ X ├
        └───┘             └───┘        └───┘


In [14]:
k=7
print("target state vector:", state_vectors[k])

circ7 = QuantumCircuit(q)
circ7.x(q[0])
circ7.cz(q[0], q[1])
circ7.x(q[0])

circ7.x(q[1])
circ7.cz(q[0], q[1])
circ7.x(q[1])

circ7.cz(q[0], q[1])

draw_state_vector(circ + circ7)

target state vector: [ 0.5 -0.5 -0.5 -0.5]
state vector after circuit: [ 0.5+0.j -0.5+0.j -0.5+0.j -0.5+0.j]
             ┌───┐┌───┐        ┌───┐           
q_0: |0>─────┤ H ├┤ X ├─■──────┤ X ├─■───────■─
        ┌───┐└───┘└───┘ │ ┌───┐└───┘ │ ┌───┐ │ 
q_1: |0>┤ H ├───────────■─┤ X ├──────■─┤ X ├─■─
        └───┘             └───┘        └───┘   


In [15]:
k=8
print("target state vector:", state_vectors[k])

circ8 = QuantumCircuit(q)
circ8.x(q[0])
circ8.x(q[1])
circ8.cz(q[0], q[1])
circ8.x(q[0])
circ8.x(q[1])

draw_state_vector(circ + circ8)

target state vector: [-0.5  0.5  0.5  0.5]
state vector after circuit: [-0.5+0.j  0.5+0.j  0.5+0.j  0.5+0.j]
                  ┌───┐┌───┐        ┌───┐
q_0: |0>──────────┤ H ├┤ X ├─■──────┤ X ├
        ┌───┐┌───┐└───┘└───┘ │ ┌───┐└───┘
q_1: |0>┤ H ├┤ X ├───────────■─┤ X ├─────
        └───┘└───┘             └───┘     


In [16]:
k=9
print("target state vector:", state_vectors[k])

circ9 = QuantumCircuit(q)

# this becomes the "encode -|00> block" or k_8
circ9.x(q[0])
circ9.x(q[1])
circ9.cz(q[0], q[1])
circ9.x(q[0])
circ9.x(q[1])

# This is k_1
circ9.cz(q[0], q[1])

draw_state_vector(circ + circ9)

target state vector: [-0.5  0.5  0.5 -0.5]
state vector after circuit: [-0.5+0.j  0.5+0.j  0.5+0.j -0.5+0.j]
                  ┌───┐┌───┐        ┌───┐   
q_0: |0>──────────┤ H ├┤ X ├─■──────┤ X ├─■─
        ┌───┐┌───┐└───┘└───┘ │ ┌───┐└───┘ │ 
q_1: |0>┤ H ├┤ X ├───────────■─┤ X ├──────■─
        └───┘└───┘             └───┘        


### A pattern emerges
Here we begin to repeat circuit combinations. E.g. $k_9 = k_8 + k_1$
<br><br>
As noted in the block above, the sign flip block for $k_8$ will now be needed for every $8\leq k < 16$.
<br><br>
So we may as well just incorporate circ8 in combination with some circ(<8) into our future circuits.

In [17]:
# alternatively for k=9
print("target state vector:", state_vectors[k])

circ9a = QuantumCircuit(q)
circ9a = circ + circ8 + circ1

draw_state_vector(circ9a)

target state vector: [-0.5  0.5  0.5 -0.5]
state vector after circuit: [-0.5+0.j  0.5+0.j  0.5+0.j -0.5+0.j]
                  ┌───┐┌───┐        ┌───┐   
q_0: |0>──────────┤ H ├┤ X ├─■──────┤ X ├─■─
        ┌───┐┌───┐└───┘└───┘ │ ┌───┐└───┘ │ 
q_1: |0>┤ H ├┤ X ├───────────■─┤ X ├──────■─
        └───┘└───┘             └───┘        


In [18]:
k=10
print("target state vector:", state_vectors[k])

circ10 = QuantumCircuit(q)
circ10 = circ + circ8 + circ2

draw_state_vector(circ10)

target state vector: [-0.5  0.5 -0.5  0.5]
state vector after circuit: [-0.5+0.j  0.5+0.j -0.5+0.j  0.5+0.j]
                  ┌───┐┌───┐        ┌───┐┌───┐   ┌───┐
q_0: |0>──────────┤ H ├┤ X ├─■──────┤ X ├┤ X ├─■─┤ X ├
        ┌───┐┌───┐└───┘└───┘ │ ┌───┐└───┘└───┘ │ └───┘
q_1: |0>┤ H ├┤ X ├───────────■─┤ X ├───────────■──────
        └───┘└───┘             └───┘                  


In [19]:
k=11
print("target state vector:", state_vectors[k])

circ11 = QuantumCircuit(q)
circ11 = QuantumCircuit(q)
circ11 = circ + circ8 + circ3

draw_state_vector(circ11)

target state vector: [-0.5  0.5 -0.5 -0.5]
state vector after circuit: [-0.5+0.j  0.5+0.j -0.5+0.j -0.5+0.j]
                  ┌───┐┌───┐        ┌───┐┌───┐   ┌───┐   
q_0: |0>──────────┤ H ├┤ X ├─■──────┤ X ├┤ X ├─■─┤ X ├─■─
        ┌───┐┌───┐└───┘└───┘ │ ┌───┐└───┘└───┘ │ └───┘ │ 
q_1: |0>┤ H ├┤ X ├───────────■─┤ X ├───────────■───────■─
        └───┘└───┘             └───┘                     


In [20]:
k=12
print("target state vector:", state_vectors[k])

circ12 = QuantumCircuit(q)
circ12 = QuantumCircuit(q)
circ12 = circ + circ8 + circ4

draw_state_vector(circ12)

target state vector: [-0.5 -0.5  0.5  0.5]
state vector after circuit: [-0.5+0.j -0.5+0.j  0.5+0.j  0.5+0.j]
                  ┌───┐┌───┐             ┌───┐        
q_0: |0>──────────┤ H ├┤ X ├─■───────────┤ X ├─■──────
        ┌───┐┌───┐└───┘└───┘ │ ┌───┐┌───┐└───┘ │ ┌───┐
q_1: |0>┤ H ├┤ X ├───────────■─┤ X ├┤ X ├──────■─┤ X ├
        └───┘└───┘             └───┘└───┘        └───┘


In [21]:
k=13
print("target state vector:", state_vectors[k])

circ13 = QuantumCircuit(q)
circ13 = QuantumCircuit(q)
circ13 = circ + circ8 + circ5

draw_state_vector(circ13)

target state vector: [-0.5 -0.5  0.5 -0.5]
state vector after circuit: [-0.5+0.j -0.5+0.j  0.5+0.j -0.5+0.j]
                  ┌───┐┌───┐             ┌───┐           
q_0: |0>──────────┤ H ├┤ X ├─■───────────┤ X ├─■───────■─
        ┌───┐┌───┐└───┘└───┘ │ ┌───┐┌───┐└───┘ │ ┌───┐ │ 
q_1: |0>┤ H ├┤ X ├───────────■─┤ X ├┤ X ├──────■─┤ X ├─■─
        └───┘└───┘             └───┘└───┘        └───┘   


In [22]:
k=14
print("target state vector:", state_vectors[k])

circ14 = QuantumCircuit(q)
circ14 = QuantumCircuit(q)
circ14 = circ + circ8 + circ6

draw_state_vector(circ14)

target state vector: [-0.5 -0.5 -0.5  0.5]
state vector after circuit: [-0.5+0.j -0.5+0.j -0.5+0.j  0.5+0.j]
                  ┌───┐┌───┐        ┌───┐┌───┐        ┌───┐        
q_0: |0>──────────┤ H ├┤ X ├─■──────┤ X ├┤ X ├─■──────┤ X ├─■──────
        ┌───┐┌───┐└───┘└───┘ │ ┌───┐└───┘└───┘ │ ┌───┐└───┘ │ ┌───┐
q_1: |0>┤ H ├┤ X ├───────────■─┤ X ├───────────■─┤ X ├──────■─┤ X ├
        └───┘└───┘             └───┘             └───┘        └───┘


In [23]:
k=15
print("target state vector:", state_vectors[k])

circ15 = QuantumCircuit(q)
circ15 = QuantumCircuit(q)
circ15 = circ + circ8 + circ7

draw_state_vector(circ15)

target state vector: [-0.5 -0.5 -0.5 -0.5]
state vector after circuit: [-0.5+0.j -0.5+0.j -0.5+0.j -0.5+0.j]
                  ┌───┐┌───┐        ┌───┐┌───┐        ┌───┐           
q_0: |0>──────────┤ H ├┤ X ├─■──────┤ X ├┤ X ├─■──────┤ X ├─■───────■─
        ┌───┐┌───┐└───┘└───┘ │ ┌───┐└───┘└───┘ │ ┌───┐└───┘ │ ┌───┐ │ 
q_1: |0>┤ H ├┤ X ├───────────■─┤ X ├───────────■─┤ X ├──────■─┤ X ├─■─
        └───┘└───┘             └───┘             └───┘        └───┘   


## Big O
Just by looking at the circuits, we start to get a sense of how the brute-force sign flip algorithm scales. <br>
In the worst cases, which correspond to high values of k, the corresponding state vectors require circuits that scale $\textbf{exponentially}$ in depth (left to right time slices).

## A general Sign-Flip function to encode a quantum state vector
This brute-force Sign-Flip algorithm uses only X and CZ gates. It effectively acts on one component of the state vector at a time.
<br><br>
Right now this will only work for N=2 qubits. But it gives you the right idea!

In [24]:
def SF_encoder(i):
    """
        Parameters:
            - i (list) A classical binary valued {-1, 1} input vector
        Returns:
            - circ (qiskit.circuit.quantumcircuit.QuantumCircuit) A quantum circuit that 
              properly encodes the input vector i
        Description:
            - convert vector to binary string, or somehow generate computatinal basis vectors    
            - Apply SF algorithm whenver there is a negative -1 in i
            - Use -1 position to indentify corresponding computational basis, sandwich qubits
              in the 0 state with X gates, and in-between apply a CZ
    """
    
    i = np.array(i)
    d = len(i) # dimesionality of input vector
    
    # Validate input
    assert math.log(d, 2)%1 == 0, "Invalid vector length. Must be 2^N"
    
    N = int(math.log(d, 2)) # number of qubits
    
    # Generate computational basis (standard basis) vectors with Dirac labeling
    Dirac_vectors = [("{:0%db}"%N).format(k) for k in range(d)]
    
    '''
    for b in Dirac_vectors:
        print(b)
    '''
    
    # create quantum register and circuit
    q = QuantumRegister(N, 'q')
    circ = QuantumCircuit(q)
    
    # always start in equal superposition
    circ.h(q[0])
    circ.h(q[1])
    
    # Find all components with a -1 factor in i (and thus our target state vector)
    indices = np.where(i == -1)[0]
    if indices.size > 0:
        for idx in indices:
            # Need to switch qubits in the 0 state so CZ will take effect
            for i, b in enumerate(Dirac_vectors[idx]):
                if b == '0':
                    circ.x(q[(N-1)-i]) # (N-1)-i is to match the qubit ordering Qiskit uses (reversed)
                    
            circ.cz(q[0], q[1]) # this is the only part that doesn't generalize for N!=2

            # And switch the flipped qubits back
            for i, b in enumerate(Dirac_vectors[idx]):
                if b == '0':
                    circ.x(q[(N-1)-i])
            
    return circ

In [25]:
# You can test the algorithm here!
k = 11
print(state_vectors[k])
cc = SF_encoder(vectors[k])
draw_state_vector(cc)

[-0.5  0.5 -0.5 -0.5]
state vector after circuit: [-0.5+0.j  0.5+0.j -0.5+0.j -0.5+0.j]
                  ┌───┐┌───┐   ┌───┐┌───┐        ┌───┐   
q_0: |0>──────────┤ H ├┤ X ├─■─┤ X ├┤ X ├──────■─┤ X ├─■─
        ┌───┐┌───┐└───┘└───┘ │ └───┘└───┘┌───┐ │ └───┘ │ 
q_1: |0>┤ H ├┤ X ├───────────■───────────┤ X ├─■───────■─
        └───┘└───┘                       └───┘           
